# SOx 배출 감소 및 석고 순도 향상을 위한 최적의 석회석 사용량

## 소개
배연 탈황(Flue Gas Desulfurization, FGD) 과정에서는 황산화물(SOx) 배출을 최대한 줄이면서, 과정의 부산물인 석고의 순도를 높이는 것이 목표입니다. 이 문서에서는 SOx 감소 효율과 석고 순도를 최적화하기 위한 석회석 사용량을 결정하는 알고리즘을 설명합니다.

## 문제 설명
주요 목표는 FGD 과정에서 사용할 최적의 석회석 사용량을 찾는 것입니다. 석회석 사용량이 증가하면 SOx 감소(SOx diff)도 증가하지만, 과도한 석회석 사용은 석고 순도가 감소할 수 있습니다.

## 알고리즘 개요
1. **데이터 수집**: `Inlet Sox`, `Outlet Sox`, `Limestone` 사용량 데이터를 수집합니다.
2. **SOx 감소 계산**: `SOx diff = Inlet Sox - Outlet Sox`를 계산합니다.
3. **최적 지점 탐색**: SOx 감소가 점차 감소하는 지점을 찾아 최적의 석회석 사용량을 결정합니다.

## 단계별 설명
1. **데이터 준비**:
    - 관련 데이터(Inlet Sox, Outlet Sox, Limestone 사용량)를 수집하고 정리합니다.
2. **SOx 감소 계산**:
    - 각 데이터 포인트에 대해 SOx diff를 계산합니다.
3. **최적 지점 탐색**:
    - SOx diff와 Limestone 사용량의 관계를 그래프로 시각화합니다.
    - SOx diff 증가율이 감소하기 시작하는 지점을 찾아 최적의 Limestone 사용량을 결정합니다.

### 2. 초기값 설정
- 입력받은 `Inlet Sox` 값을 기준으로 해당 구간에서 Limestone의 평균값을 계산합니다.
- 초기값으로 Limestone 평균값보다 약간 작은 값을 설정합니다.

### 3. 모델 예측 및 최적값 탐색
- Limestone 값을 0.1씩 증가시키며 반복합니다.
- 모델을 통해 `SOx diff` 예측값을 계산합니다.
- 직전 `SOx diff` 값과 비교하여 증가폭이 0.01 이상이면 해당 `SOx diff`와 Limestone 값을 최적의 값으로 기록하고 다음 반복을 실행합니다.
- 만약 증가폭이 0.01 이하인 경우가 3번 연속으로 발생하면 현재 `SOx diff`가 최대값 근처라고 판단하고 3번 반복할 때의 첫 번째 Limestone 값을 최적의 값으로 출력합니다.
- 만약 `SOx diff`가 감소하면 즉시 반복을 중단하고 직전의 Limestone 값을 최적의 값으로 출력합니다.

In [3]:
# Inlet Sox 구간 설정 및 평균 석회석 사용량 계산
bins = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
df['InletSox_Range'] = pd.cut(df['InletSox_Unit3'], bins=bins, labels=labels)

# 구간별 평균 석회석 사용량 계산
avg_limestone_usage = df.groupby('InletSox_Range')['Limestone_Unit3'].mean().reset_index()

# 결과 출력
print("Inlet Sox 구간\t평균 석회석 사용량 (ton)")
for _, row in avg_limestone_usage.iterrows():
    print(f"{row['InletSox_Range']}\t{row['Limestone_Unit3']:.6f}")

Inlet Sox 구간	평균 석회석 사용량 (ton)
0-100	1.783524
100-150	4.996903
150-200	6.950159
200-250	8.337030
250-300	10.267330
300+	14.374816


C:\Users\lee seung woo\AppData\Local\Temp\ipykernel_10648\3733770645.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_limestone_usage = df.groupby('InletSox_Range')['Limestone_Unit3'].mean().reset_index()


In [5]:
# Define a function to get the initial limestone based on inlet_sox
def get_initial_limestone(inlet_sox):
    if inlet_sox <= 100:
        return 1.83524-0.5
    elif inlet_sox <= 150:
        return 4.996510-0.5
    elif inlet_sox <= 200:
        return 6.843625-0.5
    elif inlet_sox <= 250:
        return 8.337030-0.5
    elif inlet_sox <= 300:
        return 10.207591-0.5
    else:
        return 14.374816-0.5

# Calculate the average power for Unit 3
average_power_unit3 = df['Power_Unit3'].mean()

In [6]:
def find_optimal_limestone_soxdiff(model, inlet_sox, power_unit3=None, threshold=0.01, increment=0.1, max_iterations=1000):
    if power_unit3 is None:
        power_unit3 = average_power_unit3
    initial_limestone = get_initial_limestone(inlet_sox)
    current_limestone = initial_limestone
    previous_sox_diff = 0
    consecutive_count = 0  # 연속으로 조건을 만족한 횟수
    first_optimal_limestone = None  # 처음으로 조건을 만족한 limestone 값

    for iteration in range(max_iterations):
        current_limestone += increment
        data = pd.DataFrame({
            'Power_Unit3': [power_unit3],
            'InletSox_Unit3': [inlet_sox],
            'Limestone_Unit3': [current_limestone]
        })
        current_sox_diff = model.predict(data)[0]

        # 디버깅을 위한 출력
        print(f"Iteration: {iteration}, Limestone: {current_limestone}, SoxDiff: {current_sox_diff}, Previous SoxDiff: {previous_sox_diff}")

        if abs(current_sox_diff - previous_sox_diff) < threshold:
            consecutive_count += 1
            if consecutive_count == 1:
                first_optimal_limestone = current_limestone
            if consecutive_count >= 3:
                break
        else:
            consecutive_count = 0

        # SoxDiff가 감소하면 중단하고 직전의 limestone 값을 최적의 값으로 저장
        if current_sox_diff < previous_sox_diff:
            return previous_limestone, previous_sox_diff

        previous_sox_diff = current_sox_diff
        previous_limestone = current_limestone

    return first_optimal_limestone, current_sox_diff

In [27]:
inlet_sox = 120 # This can be input by the user
optimal_limestone, final_sox_diff = find_optimal_limestone_soxdiff(model_soxdiff, inlet_sox)

print(f"Optimal Limestone: {optimal_limestone}")
print(f"Final SoxDiff: {final_sox_diff}")

Iteration: 0, Limestone: 4.596509999999999, SoxDiff: 120.24967504237144, Previous SoxDiff: 0
Iteration: 1, Limestone: 4.696509999999999, SoxDiff: 120.24967504237144, Previous SoxDiff: 120.24967504237144
Iteration: 2, Limestone: 4.796509999999999, SoxDiff: 120.24967504237144, Previous SoxDiff: 120.24967504237144
Iteration: 3, Limestone: 4.896509999999998, SoxDiff: 120.24967504237144, Previous SoxDiff: 120.24967504237144
Optimal Limestone: 4.696509999999999
Final SoxDiff: 120.24967504237144
